Preprocessing

The input of the model will be the image, the rest of variables will be used to delimit and make statistics and later studies. We will take Nmuons and Class as output variables, since from the image we are going to classify the primary particle and estimate the number of muons.

In [1]:
##Libraries needed
import numpy as np
import pandas as pd
import h5py 
import matplotlib.pyplot as plt
import matplotlib as mpl
import os

In [2]:

primary = "gamma"
size = 560 #detector radius, area: 1km^2

if (primary=="gamma"):
    filename_train = "photon_alt5200m_qgsii_fluka_r560m_3PMTs.h5"
    particle = "Gamma"
elif (primary=="proton"):
    filename_train = "proton_alt5200m_qgsii_fluka_r560m_3PMTs.h5"
    particle = "Proton"
elif (primary=="iron"):
    filename_train = "iron_alt5200m_qgsii_fluka_r560m_3PMTs.h5"
    particle = "Iron"


In [3]:
#Read data
f = h5py.File(filename_train, mode = "r")

group = f["data"]

#Read shower parameters
InfoDF = pd.read_hdf(filename_train, key = "info")
ID_showers = InfoDF.iloc[:,0].values
E0_train = InfoDF.iloc[:,1].values
theta0_train = InfoDF.iloc[:,2].values
Nmuons_train = InfoDF.iloc[:,3].values

#Read input for the algorithm
#X_train: channel1 -> e.m. energy, channel2 -> particles, channel3 -> muons
X_train = group[()]
Y_train = InfoDF.iloc[:,-1].values


#Close file
f.close()

In [4]:
X_train.shape

(33299, 283, 283)

In [5]:
#Saving images
E_thres = 80000 #energy threshold
X_train_E0 = X_train[E0_train>E_thres,:,:]
for i in range(len(X_train_E0)):
    try:
        fig = plt.figure(figsize=(4,4))
        plt.imshow(X_train_E0[i,:,:], alpha=0.8, cmap='Reds',norm=mpl.colors.LogNorm())
        plt.axis('off')
        filename = './data/'+str(primary)+'/'+str(primary)+'_id'+str(i)+'.jpg'
        plt.savefig(filename)
        plt.close(fig)
    except: 
        print('Error in i = '+str(i))
        pass

Error in i = 298
Error in i = 800
Error in i = 1130
Error in i = 1854
Error in i = 1946


ValueError: vmin must be less or equal to vmax

<Figure size 288x288 with 1 Axes>

ValueError: vmin must be less or equal to vmax

<Figure size 288x288 with 1 Axes>

...

<Figure size 288x288 with 1 Axes>

ValueError: vmin must be less or equal to vmax

<Figure size 288x288 with 1 Axes>

ValueError: vmin must be less or equal to vmax

<Figure size 288x288 with 1 Axes>

In [6]:
#Counting elements in each folder
import os

count = 0
dir_path = 'data/gamma'
for path in os.scandir(dir_path):
    if path.is_file():
        count += 1
print('file count gamma:', count)

count = 0
dir_path = 'data/proton'
for path in os.scandir(dir_path):
    if path.is_file():
        count += 1
print('file count proton:', count)

count = 0
dir_path = 'data/iron'
for path in os.scandir(dir_path):
    if path.is_file():
        count += 1
print('file count iron:', count)

file count gamma: 2650
file count proton: 1480
file count iron: 2272


In [7]:
#Images which fails

print(np.sum(X_train_E0[298,:,:]))
print(np.sum(X_train_E0[800,:,:]))
print(np.sum(X_train_E0[1130,:,:]))
print(np.sum(X_train_E0[1854,:,:]))
print(np.sum(X_train_E0[1946,:,:]))

0.0
0.0
0.0
0.0
0.0


The images that failed have total signal in the event 0 (sum of all elements in the matrix for that event).

In [8]:
#Split folders train val and test
import splitfolders

splitfolders.ratio('data', output="data_split", seed=1337, ratio=(.8, 0.1,0.1)) 

Copying files: 6402 files [00:03, 1833.29 files/s]
